In [50]:
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import Row
from pyspark.sql.types import *       # for datatype conversion
from pyspark.sql.functions import *   # for col() function
import pandas as pd
from pyspark.sql import SparkSession
from pyspark.ml.feature import StopWordsRemover
from pyspark.ml.feature import Tokenizer
import re
import pyspark.sql.functions as f
from pyspark.ml.feature import HashingTF, IDF, Tokenizer
from numpy import array
from math import sqrt
from pyspark.ml.clustering import KMeans


sc = SparkContext.getOrCreate()
sqlCtx = SQLContext(sc)

spark = SparkSession \
    .builder \
    .master("local") \
    .appName("Jeopardy Calculation") \
    .config("spark.executor.memory", '2g') \
    .config('spark.executor.cores', '1') \
    .config('spark.cores.max', '1') \
    .config("spark.driver.memory",'1g') \
    .getOrCreate()



In [51]:
df = sqlCtx.read.format("csv").option("header", "true").option("delimiter", "\t").option("inferSchema", "true").load("data/rspct.tsv")

In [52]:
df.cache()

DataFrame[id: string, subreddit: string, title: string, selftext: string]

In [53]:
cleaned = df.withColumn('cleaned', regexp_replace('selftext', '<lb>', ' '))

In [54]:
cleaned.show(5)

+------+--------------------+--------------------+--------------------+--------------------+
|    id|           subreddit|               title|            selftext|             cleaned|
+------+--------------------+--------------------+--------------------+--------------------+
|6d8knd|talesfromtechsupport|Remember your com...|"Hi there,  <lb>T...|"Hi there,   The ...|
|58mbft|             teenmom|"So what was Matt...|Did he ever say w...|Did he ever say w...|
|8f73s7|              Harley|      No Club Colors|Funny story. I we...|Funny story. I we...|
|6ti6re|        ringdoorbell|Not door bell, bu...|I know this is a ...|I know this is a ...|
|77sxto|               intel|Worried about my ...|"Prime95 (regardl...|"Prime95 (regardl...|
+------+--------------------+--------------------+--------------------+--------------------+
only showing top 5 rows



In [72]:
data = cleaned.withColumn("stripped", f.regexp_replace(f.col("cleaned"), "[,.'':\[\]\$&!;'""']", ""))
#tokenizer = Tokenizer(inputCol = "stripped", outputCol = "words")
#tokenized = tokenizer.transform(data)
#remover = StopWordsRemover(inputCol="words", outputCol="filtered")
#remover.transform(tokenized).show(truncate=False)
#dataStopRemoved = remover.transform(tokenized)


In [73]:
data.select("cleaned", "stripped").show(2,truncate=False)

+----------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------------+----------------------------------------------------------

In [10]:
from nltk.stem.porter import *
stemmer = PorterStemmer()


def stem(in_vec):
    out_vec = []
    for t in in_vec:
        t_stem = stemmer.stem(t)
        out_vec.append(t_stem)       
    return out_vec

from pyspark.sql.types import *
stemmer_udf = udf(lambda x: stem(x), ArrayType(StringType()))


reddit_processed = dataStopRemoved.withColumn("stemmed", stemmer_udf("filtered"))

In [9]:
#reddit_stemmed = reddit_processed.select("stemmed").rdd
#jeopardy_words = jeopardy_stemmed.flatMap(lambda r: r)
reddit_processed.show(2)

Py4JJavaError: An error occurred while calling o121.showString.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 4.0 failed 1 times, most recent failure: Lost task 0.0 in stage 4.0 (TID 17, localhost, executor driver): java.util.regex.PatternSyntaxException: Illegal character range near index 11
[''\!@#\$?-%^&*).[](><,';:]
           ^
	at java.util.regex.Pattern.error(Pattern.java:1957)
	at java.util.regex.Pattern.range(Pattern.java:2657)
	at java.util.regex.Pattern.clazz(Pattern.java:2564)
	at java.util.regex.Pattern.sequence(Pattern.java:2065)
	at java.util.regex.Pattern.expr(Pattern.java:1998)
	at java.util.regex.Pattern.compile(Pattern.java:1698)
	at java.util.regex.Pattern.<init>(Pattern.java:1351)
	at java.util.regex.Pattern.compile(Pattern.java:1028)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutableProjection.RegExpReplace_1$(Unknown Source)
	at org.apache.spark.sql.catalyst.expressions.GeneratedClass$SpecificMutabl

In [ ]:
hashingTF = HashingTF(inputCol="stemmed", outputCol="rawFeatures", numFeatures=100000)
featurizedData = hashingTF.transform(reddit_processed)
idf = IDF(inputCol="rawFeatures", outputCol="features")
idfModel = idf.fit(featurizedData)
rescaledData = idfModel.transform(featurizedData)
rescaledData.cache()



In [ ]:
training,test = rescaledData.randomSplit([0.8,0.2], seed = 1) 
kmeans = KMeans(k=50, seed=1)
clusters = kmeans.fit(training.select('features'))
transformed = clusters.transform(test)
transformed.cache()
#processedData = rescaledData.select("features")
#processedData.show()
#processedDataRDD = processedData.rdd.map
#clusters = KMeans.train(processedDataRDD, 2, maxIterations=10, initializationMode="random")

In [ ]:
transformed.groupBy('prediction').count().sort(desc("prediction")).show(50)